In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
%matplotlib inline

In [2]:
import re
import yaml
import geopandas as gpd
import requests
import shutil
import pprint
from urllib.request import urlretrieve
from datetime import datetime
from zipfile import ZipFile

# Notebook Styling 
sns.set()
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))

DATA_DIR = os.path.join('..', 'data_raw')

In [51]:
from typing import (
    Dict,
#     List,
#     Tuple,
#     Set,
#     Deque,
#     NamedTuple,
#     IO,
#     Pattern,
#     Match,
#     Text,
#     Optional,
#     Sequence,
#     Iterable,
#     Mapping,
#     MutableMapping,
#     Any,
)

In [3]:
def ensure_dir_exists(DIR_PATH):
    if not os.path.isdir(DIR_PATH):
        os.makedirs(DIR_PATH)
        
def freq_label_selector(freq):
    if freq.lower() == 'm':
        return 'Month'
    elif freq.lower() == 'q':
        return 'Quarter'
    else:
        return 'Year'

In [4]:
def get_nypd_data_table_urls():
    YAML_DIR = os.path.join('..', 'yaml_files')
    ensure_dir_exists(YAML_DIR)
    yaml_path = os.path.join(YAML_DIR, 'nypd_data_urls.yaml')
    
    if not os.path.isfile(yaml_path):
        print('Making first yaml data file')
        # Initial data load
        nyc_data_urls = {
            'nypd_shooting_historic':'https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD',
            'nypd_shooting_ytd':'https://data.cityofnewyork.us/api/views/5ucz-vwe8/rows.csv?accessType=DOWNLOAD',
            'nypd_arrest_historic':'https://data.cityofnewyork.us/api/views/8h9b-rp9u/rows.csv?accessType=DOWNLOAD',
            'nypd_arrest_ytd':'https://data.cityofnewyork.us/api/views/uip8-fykc/rows.csv?accessType=DOWNLOAD',
            'nypd_complaint_historic':'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD',
            'nypd_complaint_ytd':'https://data.cityofnewyork.us/api/views/5uac-w243/rows.csv?accessType=DOWNLOAD',
        }
        yaml.dump(data=nyc_data_urls, stream=open(yaml_path, 'w'))
    nyc_data_urls = yaml.load(stream=open(os.path.join(yaml_path), 'r'), Loader=yaml.FullLoader)
    return nyc_data_urls

In [5]:
nyc_data_urls = get_nypd_data_table_urls()

In [6]:
nyc_data_urls

{'nypd_arrest_historic': 'https://data.cityofnewyork.us/api/views/8h9b-rp9u/rows.csv?accessType=DOWNLOAD',
 'nypd_arrest_ytd': 'https://data.cityofnewyork.us/api/views/uip8-fykc/rows.csv?accessType=DOWNLOAD',
 'nypd_complaint_historic': 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD',
 'nypd_complaint_ytd': 'https://data.cityofnewyork.us/api/views/5uac-w243/rows.csv?accessType=DOWNLOAD',
 'nypd_shooting_historic': 'https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD',
 'nypd_shooting_ytd': 'https://data.cityofnewyork.us/api/views/5ucz-vwe8/rows.csv?accessType=DOWNLOAD'}

In [7]:
def get_age_of_data_pull(data_table_name: str, DATA_DIR: str = DATA_DIR, VERBOSE: bool = False) -> float:
    """ Checks to the raw_data file of the most recent data pull. If it exists, it returns
    the age of the file in days. If it doesn't exist, it returns 36500 days. """
    dir_path = os.path.join(DATA_DIR)
    file_path = os.path.join(dir_path, f"{data_table_name}.csv")
    ensure_dir_exists(dir_path)
    if (os.path.isfile(file_path)):
        file_datetime = datetime.fromtimestamp(os.path.getmtime(file_path))
        file_age = ((datetime.now() - file_datetime).seconds) / (24 * 60 * 60)
    else:
        file_age = 100 * 365
    if VERBOSE: 
        print(f"{file_age} (days)")
        print(f"{file_age / 365} (years)")
    return file_age

In [8]:
get_age_of_data_pull(data_table_name = 'nypd_shooting_historic')

0.9603472222222222

In [16]:
def pull_raw_data_table(data_table_name: str, max_days_before_update: int = 7, 
                        DATA_DIR: str = DATA_DIR) -> None:
    data_urls = get_nypd_data_table_urls()
    if data_table_name in data_urls.keys():
        file_path = os.path.join(DATA_DIR, f'{data_table_name}.csv')
        file_age = get_age_of_data_pull(data_table_name = data_table_name)
        if file_age > max_days_before_update:
            urlretrieve(data_urls[data_table_name], file_path)
            print(f"{data_table_name} data successfully downloaded")
        else:
            print(f"{data_table_name} was pulled {round(file_age, 2)} days ago.")
            print(f"Reduce the max_days_before_update parameter (currently {max_days_before_update} days) to pull now.")
    else:
        print(f'No known URL for the (alleged) data table with the name {data_table_name}.')

In [17]:
pull_raw_data_table(data_table_name = 'nypd_shooting_historic')

nypd_shooting_historic was pulled 0.96 days ago.
Reduce the max_days_before_update parameter (currently 7 days) to pull now.


In [18]:
pull_raw_data_table(data_table_name = 'nypd_shooting_ytd')

nypd_shooting_ytd was pulled 0.96 days ago.
Reduce the max_days_before_update parameter (currently 7 days) to pull now.


In [19]:
pull_raw_data_table(data_table_name = 'nypd_arrest_historic')

nypd_arrest_historic was pulled 0.96 days ago.
Reduce the max_days_before_update parameter (currently 7 days) to pull now.


In [20]:
pull_raw_data_table(data_table_name = 'nypd_arrest_ytd')

nypd_arrest_ytd was pulled 0.96 days ago.
Reduce the max_days_before_update parameter (currently 7 days) to pull now.


In [21]:
pull_raw_data_table(data_table_name = 'nypd_complaint_historic')

nypd_complaint_historic was pulled 0.94 days ago.
Reduce the max_days_before_update parameter (currently 7 days) to pull now.


In [22]:
pull_raw_data_table(data_table_name = 'nypd_complaint_ytd')

nypd_complaint_ytd data successfully downloaded


In [ ]:
arrest_dtypes = {
    'ARREST_KEY':str, 
    'PD_CD':str, 
    'PD_DESC':str, 
    'KY_CD':str, 
    'ARREST_PRECINCT':str,
    'JURISDICTION_CODE':str
}

In [70]:
def get_nypd_data_table_dtypes(data_table_name: str) -> Dict:
    YAML_DIR = os.path.join('..', 'yaml_files')
    ensure_dir_exists(YAML_DIR)
    yaml_path = os.path.join(YAML_DIR, 'nypd_data_table_dtypes.yaml')
    
    if not os.path.isfile(yaml_path):
        print('Making first yaml data file')
        # Initial data load
        nyc_data_dtypes = {
            'arrest_dtypes':{
                'ARREST_KEY':str, 
                'PD_CD':str, 
                'PD_DESC':str, 
                'KY_CD':str, 
                'ARREST_PRECINCT':str,
                'JURISDICTION_CODE':str
            }
        }        
        yaml.dump(data=nyc_data_dtypes, stream=open(yaml_path, 'w+'))
    nyc_data_dtypes = yaml.load(stream=open(os.path.join(yaml_path), 'r'), Loader=yaml.FullLoader)
    dtype_map = get_nypd_data_table_dtype_map(data_table_name=data_table_name)
    return nyc_data_dtypes[dtype_map[data_table_name]]

In [71]:
def get_nypd_data_table_dtype_map(data_table_name: str) -> Dict:
    YAML_DIR = os.path.join('..', 'yaml_files')
    ensure_dir_exists(YAML_DIR)
    yaml_path = os.path.join(YAML_DIR, 'nypd_data_table_dtype_map.yaml')
    
    if not os.path.isfile(yaml_path):
        print('Making first yaml data file')
        # Initial data load
        nyc_table_dtype_map = {
            'nypd_arrest_historic':'arrest_dtypes',
            'nypd_arrest_ytd':'arrest_dtypes',
            'nypd_complaint_historic':'complaint_dtypes',
            'nypd_complaint_ytd':'complaint_dtypes',
            'nypd_shooting_historic':'shooting_dtypes',
            'nypd_shooting_ytd':'shooting_dtypes',                
        }
        yaml.dump(data=nyc_table_dtype_map, stream=open(yaml_path, 'w+'))
    nyc_table_dtype_map = yaml.load(stream=open(os.path.join(yaml_path), 'r'), Loader=yaml.FullLoader)
    return nyc_table_dtype_map

In [72]:
def load_data_table(data_table_name: str, DATA_DIR: str = DATA_DIR) -> pd.DataFrame:
    data_urls = get_nypd_data_table_urls()
    if data_table_name in data_urls.keys():        
        file_path = os.path.join(DATA_DIR, f'{data_table_name}.csv')
        if os.path.isfile(file_path):
            table_dtypes = get_nypd_data_table_dtypes(data_table_name)
            return pd.read_csv(file_path, dtype=table_dtypes)
        else:
            print(f"{data_table_name} hasn't been pulled. Call pull_raw_data_table() then retry.")
    else:
        print(f'No known URL or file for the (alleged) data table with the name {data_table_name}.')    

In [6]:
nyc_data_urls

{'nypd_arrest_historic': 'https://data.cityofnewyork.us/api/views/8h9b-rp9u/rows.csv?accessType=DOWNLOAD',
 'nypd_arrest_ytd': 'https://data.cityofnewyork.us/api/views/uip8-fykc/rows.csv?accessType=DOWNLOAD',
 'nypd_complaint_historic': 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD',
 'nypd_complaint_ytd': 'https://data.cityofnewyork.us/api/views/5uac-w243/rows.csv?accessType=DOWNLOAD',
 'nypd_shooting_historic': 'https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD',
 'nypd_shooting_ytd': 'https://data.cityofnewyork.us/api/views/5ucz-vwe8/rows.csv?accessType=DOWNLOAD'}

In [64]:
nyc_data_dtypes = get_nypd_data_table_dtypes(data_table_name='nypd_arrest_historic')
nyc_data_dtypes

{'nypd_arrest_historic': 'arrest_dtypes', 'nypd_arrest_ytd': 'arrest_dtypes', 'nypd_complaint_historic': 'complaint_dtypes', 'nypd_complaint_ytd': 'complaint_dtypes', 'nypd_shooting_historic': 'shooting_dtypes', 'nypd_shooting_ytd': 'shooting_dtypes'}


{'ARREST_KEY': str,
 'ARREST_PRECINCT': str,
 'JURISDICTION_CODE': str,
 'KY_CD': str,
 'PD_CD': str,
 'PD_DESC': str}

In [59]:
get_nypd_data_table_dtype_map(data_table_name = 'nypd_arrest_historic')

Making first yaml data file


{'nypd_arrest_historic': 'arrest_dtypes',
 'nypd_arrest_ytd': 'arrest_dtypes',
 'nypd_complaint_historic': 'complaint_dtypes',
 'nypd_complaint_ytd': 'complaint_dtypes',
 'nypd_shooting_historic': 'shooting_dtypes',
 'nypd_shooting_ytd': 'shooting_dtypes'}

In [84]:
DATA_DIR

'../data_raw'

In [86]:
def preprocess_arrest_data(DATA_DIR: str = DATA_DIR) -> None:
    file_out_path = os.path.join(DATA_DIR, '..', 'data_preprocessed', 'nypd_arrest.feather')
    
    nypd_arrest_historic_df = load_data_table(data_table_name = 'nypd_arrest_historic')
    nypd_arrest_ytd_df = load_data_table(data_table_name = 'nypd_arrest_ytd')
    
    nypd_arrest_ytd_df.columns = nypd_arrest_historic_df.columns
    nypd_arrest_df = pd.concat([nypd_arrest_historic_df, nypd_arrest_ytd_df])
    nypd_arrest_df.reset_index(drop=True, inplace=True)
    
    nypd_arrest_df['ARREST_DATE'] = pd.to_datetime(nypd_arrest_df['ARREST_DATE'], format='%m/%d/%Y')
    nypd_arrest_df.to_feather(file_out_path)

In [89]:
def load_preprocessed_arrest_data(DATA_DIR: str = DATA_DIR) -> pd.DataFrame:
    file_path = os.path.join(DATA_DIR, '..', 'data_preprocessed', 'nypd_arrest.feather')
    if os.path.isfile(file_path):
        return pd.read_feather(file_path)
    else:
        print('No preprocessed arrest file to load')        

In [90]:
nypd_arrest_df = load_preprocessed_arrest_data()

In [91]:
nypd_arrest_df.head(2)

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,144026181,2015-06-26,639,AGGRAVATED HARASSMENT 2,361,OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV,PL 2403002,M,Q,102,0,45-64,M,WHITE HISPANIC,1031076.0,193779.0,40.698440,-73.831130,POINT (-73.83112953899997 40.69843969400005)
1,144507595,2015-07-14,969,"TRAFFIC,UNCLASSIFIED INFRACTION",881,OTHER TRAFFIC INFRACTION,VTL051101A,M,M,10,3,25-44,M,WHITE HISPANIC,984791.0,209846.0,40.742664,-73.998049,POINT (-73.99804910799998 40.74266360800004)


In [93]:
nypd_historic_df = load_data_table(data_table_name = 'nypd_shooting_historic')
nypd_ytd_df = load_data_table(data_table_name = 'nypd_shooting_ytd')

KeyError: 'shooting_dtypes'

In [95]:
nypd_historic_df = pd.read_csv(os.path.join(DATA_DIR, 'nypd_complaint_historic.csv'))
nypd_ytd_df = pd.read_csv(os.path.join(DATA_DIR, 'nypd_complaint_ytd.csv'))

/home/matt/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/matt/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (8,16,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [96]:
nypd_historic_df.head(2)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,325341655,02/11/2015,15:00:00,NaN,NaN,73.0,02/11/2015,359,OFFENSES AGAINST PUBLIC ADMINI,749.0,VIOLATION OF ORDER OF PROTECTI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,TILDEN,405,1009656.0,181287.0,25-44,BLACK,M,NaN,40.664239,-73.908425,"(40.664239422, -73.908425011)",PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M
1,393816841,03/17/2012,10:30:00,03/17/2012,11:00:00,69.0,03/17/2012,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1014035.0,174133.0,NaN,NaN,NaN,NaN,40.644590,-73.892672,"(40.644589618, -73.892672426)",PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F


In [98]:
nypd_ytd_df.shape

(196624, 36)

In [99]:
nypd_historic_df.shape

(6983207, 35)

In [101]:
set(list(nypd_historic_df.columns)) - set(list(nypd_ytd_df.columns))

set()

In [102]:
set(list(nypd_ytd_df.columns)) - set(list(nypd_historic_df.columns)) 

{'New Georeferenced Column'}

In [104]:
nypd_ytd_df = nypd_ytd_df[nypd_historic_df.columns].copy()

In [105]:
orig_col_order = list(nypd_historic_df.columns)
nypd_df = pd.concat([nypd_historic_df, nypd_ytd_df], sort=True)
nypd_df = nypd_df[orig_col_order].copy()
nypd_df.reset_index(drop=True, inplace=True)

In [ ]:
nypd_df['CMPLNT_FR_DT_orig'] = nypd_df['CMPLNT_FR_DT']
nypd_df['CMPLNT_TO_DT_orig'] = nypd_df['CMPLNT_TO_DT']

In [108]:
nypd_df['CMPLNT_FR_DT'] = nypd_df['CMPLNT_FR_DT'] + ' ' + nypd_df['CMPLNT_FR_TM']
nypd_df['CMPLNT_TO_DT'] = nypd_df['CMPLNT_TO_DT'] + ' ' + nypd_df['CMPLNT_TO_TM']

In [112]:
nypd_df['CMPLNT_FR_DT'] = pd.to_datetime(nypd_df['CMPLNT_FR_DT'], format='%m/%d/%Y %H:%M:%S')
nypd_df['CMPLNT_TO_DT'] = pd.to_datetime(nypd_df['CMPLNT_FR_DT'], format='%m/%d/%Y %H:%M:%S')
nypd_df['RPT_DT'] = pd.to_datetime(nypd_df['RPT_DT'], format='%m/%d/%Y')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1018-12-10 17:10:00

In [111]:
nypd_df.head(2)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,325341655,02/11/2015 15:00:00,15:00:00,NaN,NaN,73.0,02/11/2015,359,OFFENSES AGAINST PUBLIC ADMINI,749.0,VIOLATION OF ORDER OF PROTECTI,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,TILDEN,405,1009656.0,181287.0,25-44,BLACK,M,NaN,40.664239,-73.908425,"(40.664239422, -73.908425011)",PATROL BORO BKLYN NORTH,NaN,<18,BLACK,M
1,393816841,03/17/2012 10:30:00,10:30:00,03/17/2012 11:00:00,11:00:00,69.0,03/17/2012,344,ASSAULT 3 & RELATED OFFENSES,114.0,OBSTR BREATH/CIRCUL,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1014035.0,174133.0,NaN,NaN,NaN,NaN,40.644590,-73.892672,"(40.644589618, -73.892672426)",PATROL BORO BKLYN SOUTH,NaN,45-64,BLACK,F


In [97]:
nypd_ytd_df.head(2)

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,428569034,63,BROOKLYN,01/06/2020,00:00:00,01/06/2020,07:00:00,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,107,FELONY,FRONT OF,BURGLARY,NaN,PATROL BORO BKLYN SOUTH,233.0,"BURGLARY,TRUCK NIGHT",OTHER,01/06/2020,NaN,NaN,NaN,NaN,NaN,25-44,BLACK HISPANIC,M,1002926,168402,40.628890,-73.932719,"(40.62888963700004, -73.93271908899999)",POINT (-73.93271908899999 40.62888963700004)
1,730098953,19,MANHATTAN,01/06/2020,22:00:00,01/06/2020,23:00:00,COMPLETED,NaN,NaN,0.0,N.Y. POLICE DEPT,341,MISDEMEANOR,NaN,PETIT LARCENY,NaN,PATROL BORO MAN NORTH,349.0,"LARCENY,PETIT OF LICENSE PLATE",STREET,01/07/2020,NaN,NaN,NaN,NaN,NaN,45-64,BLACK,F,999296,224166,40.781956,-73.945672,"(40.78195554300004, -73.94567197099997)",POINT (-73.94567197099997 40.78195554300004)


In [106]:
nypd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7179831 entries, 0 to 7179830
Data columns (total 35 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CMPLNT_NUM         int64  
 1   CMPLNT_FR_DT       object 
 2   CMPLNT_FR_TM       object 
 3   CMPLNT_TO_DT       object 
 4   CMPLNT_TO_TM       object 
 5   ADDR_PCT_CD        float64
 6   RPT_DT             object 
 7   KY_CD              int64  
 8   OFNS_DESC          object 
 9   PD_CD              float64
 10  PD_DESC            object 
 11  CRM_ATPT_CPTD_CD   object 
 12  LAW_CAT_CD         object 
 13  BORO_NM            object 
 14  LOC_OF_OCCUR_DESC  object 
 15  PREM_TYP_DESC      object 
 16  JURIS_DESC         object 
 17  JURISDICTION_CODE  float64
 18  PARKS_NM           object 
 19  HADEVELOPT         object 
 20  HOUSING_PSA        object 
 21  X_COORD_CD         float64
 22  Y_COORD_CD         float64
 23  SUSP_AGE_GROUP     object 
 24  SUSP_RACE          object 
 25  SUSP_SEX          